# LA's COVID-19 Reopening / Recovery Indicators 

The NY Times wrote about rushed reopenings being [a global trial-and-error experiment](https://www.nytimes.com/2020/05/07/world/europe/coronavirus-reopening-costs.html) and a further analysis showing how US states reopening [fail to meet the WH guidelines](https://www.nytimes.com/interactive/2020/05/07/us/coronavirus-states-reopen-criteria.html). Vox has written about [the importance of COVID testing indicators](https://www.vox.com/2020/5/8/21249880/coronavirus-testing-covid-reopening-economy-lockdowns-social-distancing) as a prerequisite for reopening. 

While we wait for LA's own reopening indicators, we can still see how LA stacks up against NY and Chicago's indicators. [NY State's Reopening Indicators](https://www.nytimes.com/2020/05/04/nyregion/coronavirus-reopen-cuomo-ny.html) were based heavily in part by the White House [Opening Up America Again gating criteria](https://www.whitehouse.gov/wp-content/uploads/2020/04/Guidelines-for-Opening-Up-America-Again.pdf), and Chicago outlined even [tougher benchmarks than the IL statewide strategy](https://www.chicagotribune.com/coronavirus/ct-coronavirus-chicago-reopening-lightfoot-20200508-ztpnouwexrcvfdfcr2yccbc53a-story.html). In terms of outbreak severity, [LA and Chicago come right after NYC](https://www.nytimes.com/2020/05/09/us/coronavirus-chicago.html).

## [White House Gating Criteria](https://www.whitehouse.gov/wp-content/uploads/2020/04/Guidelines-for-Opening-Up-America-Again.pdf)
### Symptoms
* Downward trajectory of influenza-like illnesses (ILI) reported within a 14-day period **and**
* Downward trajectory of COVID-like syndromic cases reported within a 14-day period

### Cases
* Downward trajectory of documented cases within a 14-day period **or**
* Downward trajectory of positive tests as a percent of total tests within a 14-day period (flat or increasing volume of tests) 

### Hospitals
* Treat all patients without crisis care **and**
* Robust testing program in place for at-risk healthcare workers, including emerging antibody testing

| NY | Chicago |
| --- | --- |
| At least 14 days decline in total net hospitalizations on a 3-day rolling average **or** <BR> Less than 15 net new total hospitalizations on a 3-day rolling average </BR> | At least 14 days decline in new cases, on average 
| At least 14 days decline in total net deaths on a 3-day rolling average **or** <BR> Less than 5 new deaths on a 3-day rolling average </BR> | At least 14 days decline in of case rate (per capita?) |
| At least 30% total hospital and ICU beds available  | At least 14 days decline in rate of new cases (per capita?)
| At least 90 days of PPE stockpiled  | Stable or declining rates of hospitalizations  
| Capacity to conduct 30 tests per 1,000 residents per month  | Stable or declining rates of ICU admissions 
| 30 working contact tracers per 100,000 residents | Stable or declining rates of ICU deaths
| Rural regions: Fewer than 2 new COVID patients admitted per 100,000 residents per day | Share of positive results are decreasing below 15% of those tested in the community 
|   | Fewer than 1,800 coronavirus patients in hospital beds 
|   | Fewer than 600 coronavirus patients in ICU beds
|   | Fewer than 450 coronavirus patients on ventilators
|   | Test 135,000 per month, equivalent to 4,500 people per day

In [1]:
import pandas as pd
import pytz
import utils

from datetime import date, datetime, timedelta
from IPython.display import Markdown

US_COUNTY_URL = "http://lahub.maps.arcgis.com/sharing/rest/content/items/782ca660304a4bdda1cc9757a2504647/data"
LA_CITY_URL = "https://lahub.maps.arcgis.com/sharing/rest/content/items/7175fba373f541a7a19df56b6a0617f4/data"
TESTING_URL = "http://lahub.maps.arcgis.com/sharing/rest/content/items/158dab4a07b04ecb8d47fea1746303ac/data"
HOSPITAL_URL = "http://lahub.maps.arcgis.com/sharing/rest/content/items/3da1eb3e13a14743973c96b945bd1117/data"
CROSSWALK_URL = (
    "https://raw.githubusercontent.com/CityOfLosAngeles/aqueduct/master/dags/public-health/covid19/"
    "msa_county_pop_crosswalk.csv"
)

# Default parameters
county_state_name = "Los Angeles, CA"
state_name = "California"
msa_name = "Los Angeles-Long Beach-Anaheim, CA"
time_zone = "US/Pacific"
start_date = "4/1/20"
yesterday_date = (
    (datetime.today()
                .astimezone(pytz.timezone(f'{time_zone}'))
                .date()
        - timedelta(days=1)
    ).strftime("%-m/%-d/%y")
)
today_date = (
    datetime.today()
             .astimezone(pytz.timezone(f'{time_zone}'))
             .date()
             .strftime("%-m/%-d/%y")
)

## Summary of Indicators

In [ ]:
display(Markdown(f"#### Indicators as of {yesterday_date}:"))

In [ ]:
# Indicator parameters
test_lower_bound = 10_000
test_upper_bound = 16_667
positive_bound = 0.10



# New cases


# Daily Testing    
def testing_meet_indicators(yesterday_date, test_lower_bound, test_upper_bound):
    """
    Returns red/green/blue depending on how well benchmark is met
    """
    df = utils.testing_lacity(start_date, "daily", test_lower_bound, test_upper_bound)
    
    test_indicator = df[df.Date==yesterday_date].iloc[0]["Performed"]
    
    if test_indicator < test_lower_bound:
        return "red"
    elif test_indicator >= test_lower_bound:
        return "green"
    elif test_indicator >= test_upper_bound:
        return "blue"

# Positive Tests
def positive_share_meet_indicators(yesterday_date, positive_bound):
    """
    Returns red/green depending on if benchmark was met last week
    """
    df = utils.positive_tests_lacity(start_date)
    
    positive_indicator = df[df.week == df.week.max()].iloc[0]["pct_positive"]
    
    if positive_indicator < positive_bound:
        return "green"
    elif positive_indicator >= positive_bound:
        return "red"

In [ ]:
test_indicator_color = testing_meet_indicators(yesterday_date, test_lower_bound, test_upper_bound)
positive_indicator_color = positive_share_meet_indicators(yesterday_date, positive_bound)

In [ ]:
display(Markdown(f"<span style='color:{test_indicator_color}'> Daily Testing </span>"))
display(Markdown(f"<span style='color':{positive_indicator_color}'> Share of Positive Test Results </span>"))

## Case Indicators

In [2]:
la_county = utils.county_case_charts(county_state_name, start_date)
#la_city = utils.case_indicators_lacity("4/15/20")
ca = utils.state_case_charts(state_name, "3/1/20")
la_msa = utils.msa_case_charts("Los Angeles", "3/1/20")

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

In [3]:
county = utils.county_case_indicators(county_state_name, start_date)
state = utils.state_case_indicators(state_name, "3/1/20")
msa = utils.msa_case_indicators("Los Angeles", "3/1/20")

## Testing Indicators

In [ ]:
daily_tests = utils.testing_lacity(start_date, 'daily', 10_000, 16_667)
#monthly_tests = utils.testing_lacity(start_date, 'monthly', 300_000, 500_000)
positive_tests = utils.positive_tests_lacity(start_date)

In [ ]:
positive_tests[positive_tests.week == positive_tests.week.max()] 

## Hospital Capacity Indicators

In [ ]:
hospital = utils.hospital_capacity_lacity("4/1/20")

# Markdown Red/ Green Indicator 
* The next cell displays in the color red or green 

In [ ]:
def red_green():
    """
    returns a random "red" or "green" text in markdown.
    """
    return np.random.choice(['green', 'red'])

color = red_green()
print(color)
display(Markdown(f'<span style="color:{color}">some *blue* text</span>.'))

In [ ]:
"""To convert to HTML: 
jupyter nbconvert --to html --no-input --no-prompt my-notebook.ipynb
"""